# Source:
SEIR Algorithm to model evolution of Coronavirus based on: 
<br> <br>
K. Prem et al., "The effect of control strategies to reduce social mixing on outcomes of the COVID-19 epidemic in Wuhan, China: a modelling study," The Lancet Public Health, 2020, doi: 10.1016/s2468-2667(20)30073-6.
<br> 
# Model:
SEIR Stands for: <br>
S - Susceptible <br>
E - Exposed <br>
I - Infected <br>
R - Removed <br>
<br>
Population has been devided into age groups spanning five years, with the last group collecting all 75+ year olds. Each age group $$i$$ can be modelled separatelly.

The model is driven with the following equations: <br>
$$ N(t) = S(t) + E(t) + I(t) + R(t) $$ 
$$ S_{i,t+1} = S_{i,t} -\beta S_{i,t} \sum_{j = 1}^{n} C_{i,j}I_{j,t}^{c} - \alpha\beta \sum_{j = 1}^{n} C_{i,j}I_{j,t}^{sc} $$
$$ E_{i,t+1} = \beta \sum_{j = 1}^{n} C_{i,j}I_{j,t}^{c} + \alpha\beta \sum_{j = 1}^{n} C_{i,j}I_{j,t}^{sc} - (1-\kappa)E_{i,t}  $$
$$ I_{j,t+1}^{c} = \rho_{i} \kappa E_{i,t} + (1+\gamma)I_{j,t}^{c}$$
$$ I_{j,t+1}^{sc} = (1-\rho_{i}) \kappa E_{i,t} + (1+\gamma)I_{j,t}^{sc}$$
$$ R_{i,t+1} = R_{i,t} + \gamma I_{j,t+1}^{c} + \gamma I_{j,t+1}^{sc} $$ 
<br> 

## Parameters:
$ \beta $ Scaled to the value of $R_{0}$ <br>
$ C_{i,j} $ Contact parameter between different age groups <br>
$ \kappa = 1-\exp(\frac{-1}{d_{L}}) $  is the daily probability of an exposed individual becoming infectious (with $d_{L}$ being
the average incubation period) <br>
$ \gamma = 1-\exp(\frac{-1}{d_{I}}) $ is the daily probability that an infected individual recovers when the average duration of infection is $d_{I}$ <br> 
$\rho$ is probability of the infected to have symptoms, while $(1-\rho)$ is the probability to be asymptomatic<br>
$ \alpha $ - probability of getting infected from subclinical<br> 

## Numerical values implemented in the paper
$$ R_{0} = 2.2 $$
$$ d_{L} = 6.4 $$
$$ d_{I} = \{ 3, 7 \} $$
$$ I_{0} = \{ 200, 2000 \} $$
$$ \rho_{i} = \{ 0, 0.4 \} \textrm{for} i \le 4 $$ 
$$ \rho_{i} = \{ 0, 0.8 \} \textrm{for} i > 4 $$ 
$$ \alpha = 0.25 $$

In [2]:
import numpy as np
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt

In [3]:
## Import population data
age_data_url =  'https://raw.githubusercontent.com/kieshaprem/covid19-agestructureSEIR-wuhan-social-distancing/master/data/wuhanpop.csv';
age_strc = pd.read_csv(age_data_url)

interaction_file = 'C://Users//Aleksander//PycharmProjects//coronus-ml2//notebooks//marian//data//contacts_all.csv';
Cmat = pd.read_csv(interaction_file); # also works for Rds
Cmat = Cmat.iloc[:,1:].to_numpy();
#Cmat = Cmat.iloc[:,1:-1].to_numpy(); # convert to matrix

In [4]:
## Initialise parameters
# Model parameters
R0 = 2.2;
dL = 6.4;
kappa = 1-np.exp(-1/dL);
dI = 7;
gamma = 1-np.exp(-1/dI);
I0 = 200;
rho = np.zeros([16,1]);
rho[0:3] = 0.4;
rho[3:-1] = 0.8;
alpha = 0.25;
beta = R0*gamma;
# Assume outbreak started on 10th of January
n_days = 40;


# Population
N = 11e6;
S = np.zeros([16,n_days]);
E = np.zeros([16,n_days]);
Ic = np.zeros([16,n_days]);
Isc = np.zeros([16,n_days]);
R = np.zeros([16,n_days]);

Isc[:,0] = np.round(I0*age_strc.propage.to_numpy());
S[:,0] =  np.round(N*age_strc.propage.to_numpy())-Isc[:,0];


In [5]:
# Explore the outbreak
for day in range(1,n_days):
    for grp in range(0, 16):
        S[grp, day] = S[grp, day-1]-beta*S[grp, day-1]*np.dot(Cmat[grp,:], Ic[:, day-1])-alpha*beta*S[grp, day-1]*np.dot(Cmat[grp,:], Isc[:, day-1])
        E[grp, day] = beta*np.dot(Cmat[grp,:], Ic[:, day-1])+alpha*beta*np.dot(Cmat[grp,:], Isc[:, day-1])-(1-kappa)*E[grp, day-1];
        Ic[grp, day] = rho[grp]*kappa*E[grp, day-1]+(1-gamma)*Ic[grp, day-1];
        Isc[grp, day] = (1-rho[grp])*kappa*E[grp, day-1]+(1-gamma)*Isc[grp, day-1];
        R[grp, day] = R[grp, day-1] + gamma*Ic[grp, day] + gamma*Isc[grp, day];

In [6]:
# Save data to pandas frame
epi_data = {'Susceptible': np.sum(S,0),
           'Exposed': np.sum(E,0),
           'Infected': np.sum(Ic,0)+np.sum(Isc,0),
           'Removed': np.sum(E,0)}

epidemy = pd.DataFrame(epi_data, columns = ['Susceptible', 'Exposed', 'Infected', 'Removed'])
print(epidemy)

     Susceptible    Exposed    Infected    Removed
0   1.099980e+07   0.000000  201.000000   0.000000
1  -6.597823e+07  91.875358  174.242458  91.875358
2   4.687197e+08   1.059559  164.337136   1.059559
3  -4.817557e+09  90.004360  142.613501  90.004360
4   4.759158e+10   2.075823  136.648044   2.075823
5  -5.871513e+11  88.403453  118.757447  88.403453
6   6.436440e+12   3.057867  115.736178   3.057867
7  -8.436138e+13  87.027057  100.771470  87.027057
8   9.696074e+14   4.012686   99.945431   4.012686
9  -1.318983e+16  85.837655   87.220939  85.837655
10  1.575882e+17   4.945643   88.026722   4.945643
11 -2.218991e+18  84.804351   77.023830  84.804351
12  2.753549e+19   5.860840   79.037602   5.860840
13 -4.009167e+20  83.901690   69.363748  83.901690
14  5.157520e+21   6.761397   72.266672   6.761397
15 -7.742187e+22  83.108682   63.624448  83.108682
16  1.028626e+24   7.649676   67.176687   7.649676
17 -1.585061e+25  82.408018   59.340547  82.408018
18  2.164729e+26   8.527456   6

In [7]:
print(beta)
print(gamma)
print(kappa)

0.2928686205496005
0.1331221002498184
0.14465467269257748


In [8]:
grp = 4
day = 1
S[grp, day] = S[grp, day-1]-beta*S[grp, day-1]*np.dot(Cmat[grp,:], Ic[:, day-1])-alpha*beta*S[grp, day-1]*np.dot(Cmat[grp,:], Isc[:, day-1])
print(S[grp, day])
print(alpha*beta*np.dot(Cmat[grp,:], Isc[:, day-1]))
print(beta*np.dot(Cmat[grp,:], Ic[:, day-1]))
print(Cmat[grp,:])



-11754937.869245917
13.840387157191984
0.0
[0.13022835 0.16493219 0.2443277  1.97749436 3.98260418 1.31092039
 0.75788067 0.60057643 0.51261936 0.56913329 0.23529689 0.21701058
 0.07789089 0.05209109 0.05100802 0.03978713]
